In [ ]:
from qiskit import (
    QuantumCircuit,
    QuantumRegister,
    ClassicalRegister,
    AncillaRegister,
    IBMQ,
    Aer,
    __qiskit_version__,
)
from qiskit.circuit.library import QFT, GraphState, GroverOperator
from qiskit.qasm import pi
import numpy as np

from src.utils import *
from qiskit_nature.drivers import Molecule

from src.utils import get_examplary_max_cut_qp
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.circuit.library import RealAmplitudes, TwoLocal
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.compiler import transpile

In [ ]:
qp = get_examplary_max_cut_qp(3)
sim = QuantumInstance(
    backend=Aer.get_backend("qasm_simulator"), shots=1024, seed_simulator=123
)

In [ ]:
ansatz = TwoLocal(3, "ry", "cx", "full", reps=1)
# ansatz = RealAmplitudes(3, reps=2)
ansatz.decompose().draw(output="mpl", filename="flow_0")

In [ ]:
qc = ansatz.assign_parameters([np.pi, np.pi, np.pi, np.pi, np.pi, np.pi])
qc.decompose().draw(output="mpl", filename="flow_1")

In [ ]:
qc_2 = transpile(
    qc, basis_gates=["id", "rz", "sx", "x", "cx", "reset"], optimization_level=1
)
qc_2.draw(output="mpl", filename="flow_2")

In [ ]:
c_map = select_c_map("ibm", True, 3)
qc_3 = get_compiled_circuit(qc, opt_level=1, c_map=c_map)
qc_3.draw(output="mpl", filename="flow_3")

In [ ]:
ibm_native_gates = FakeMontreal().configuration().basis_gates
ibm_native_gates

In [ ]:
vqe = VQE(ansatz, optimizer=SLSQP(), quantum_instance=sim)
vqe_optimizer = MinimumEigenOptimizer(vqe)
vqe_result = vqe_optimizer.solve(qp)
qc = vqe.get_optimal_circuit()

qc.name = "VQE"

In [ ]:
vqe.get_optimal_vector()